In [64]:
import pandas as pd
import re


In [65]:
# Load CSV
path = r"C:\Users\joacosta\Work\Beumer\MTN6\Extracted\BGFusion\11-21-2025\HistoryAlarms_November 25.csv"
df = pd.read_csv(
    path,
    sep=";",
    skiprows=1
)

# --- Clean up chute name ---
def extract_chute(part_name):
    # Find the CHUxxx inside the string
    match = re.search(r"(CHU\d+)", part_name)
    return match.group(1) if match else None

df["Chute"] = df["Part name"].apply(extract_chute)

def clean_duration(x):
    if isinstance(x, str):
        x = x.replace('="', '').replace('"', '')
    return x

# --- Parse Duration into timedelta ---
df["Duration"] = df["Duration"].apply(clean_duration)
df["Duration"] = pd.to_timedelta(df["Duration"])

def load_chute_mapping():
    mapping_path = r"C:\Users\joacosta\Work\Beumer\MTN6\Python\Amazon_MTN6_Analytics\data\MTN6_Destination_Mapping.xlsx"

    # Load the Excel file as strings
    map_df = pd.read_excel(mapping_path, dtype=str)

    # Standard clean strings
    for col in map_df.columns:
        map_df[col] = map_df[col].str.strip()

    # We only need two columns for this task:
    #   Beumer Name (e.g., CHU731)
    #   Amazon (e.g., A-12)
    return map_df[["Beumer", "Amazon"]]


# Load mapping
mapping_df = load_chute_mapping()

# Merge mapping into the alarm dataframe
df = df.merge(mapping_df, left_on="Chute", right_on="Beumer", how="left")

# Rename Amazon column for clarity
# --- Group by Chute ---
grouped = df.groupby("Chute").agg(
    jam_count = ("Chute", "count"),
    total_duration = ("Duration", "sum"),
    average_duration = ("Duration", "mean"),
    max_duration = ("Duration", "max"),
    amazon_name=("Amazon", "first")   # add Amazon name to output
)

# Optional: Sort by total duration (longest total jams)
grouped = grouped.sort_values(by="total_duration", ascending=False)

# --- Format durations as HH:MM:SS ---
def format_hhmmss(td):
    if pd.isna(td):
        return ""
    total_seconds = int(td.total_seconds())

    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60

    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

for col in ["total_duration", "average_duration", "max_duration"]:
    grouped[col] = grouped[col].apply(format_hhmmss)


In [66]:
# Reset index so Chute becomes a column called "Beumer"
grouped = grouped.reset_index().rename(columns={
    "Chute": "Beumer",
    "amazon_name": "Amazon",
    "jam_count": "Jam Count",
    "total_duration": "Total Duration",
    "average_duration": "Average Duration",
    "max_duration": "Max Duration"
})

# Reorder columns
grouped = grouped[
    ["Beumer", "Amazon", "Jam Count", "Total Duration", "Average Duration", "Max Duration"]
]


In [67]:
grouped.to_clipboard(index=False)